# Stochastic model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

The following models will need the same rates and functions for events. Here, the temperature and rainfall dependencies only account for the competition in the larval stage.

In [ ]:
def allee(M, Ms):
    return (M + Ms) / (1 + 0.1 *M + 0.01 * Ms)

def release1(M, n):
    return n*M + 2000

def competition1(K0, Kh, precip):
    return 1/(K0 + Kh *  precip)

def competition2(K0, Kh, preciptemp):
    return rsp/deltaA * (K0 + Kh * (T/298)*np.exp(0.05*(1 - H/100)))

In [ ]:

init_egg = 10  # Starting egg population
init_l = 50 # Starting larval population
init_p = 20 # Starting pupal population
init_f0 = 100 # Starting female population
init_ff = 0 # Starting fertilised female pop
init_fs = 0 # Starting strerile female pop
init_male = 100 # Starting adult population
init_sterile = 0 # Starting sterile male population
max_time = 500.0     # Maximum simulation time
comp_larvae = 0.0001 # Competition coefficient for larvae
n_rel = 0
n_egg = 64

T_max = 365*2
pop = np.zeros((T_max, 8))
pop[0, :] = [init_egg, init_l, init_p, init_f0, init_ff, init_fs, init_male, init_sterile]

## Full mosquito life cycle

In [ ]:
# Parameters
birth = 0.5  # Birth rate per adult
mu = 0.5    # rate at which egg becomes a female
egg_death = 0.2     # Death rate per egg
larva_death = 0.1   # Death rate per larva
pupa_death = 0.05   # Death rate per pupa
female_death = 0.091  # Death rate per adult
male_death = 0.182    # Death rate per adult
sterile_death = male_death*1.2   # Death rate per sterile
transi_el = 0
transi_lp = 0
transi_pa = 0

In [ ]:
T_max = 365*2
pop = np.zeros((T_max, 8))
pop = [init_egg, init_l, init_p, init_f0, init_ff, init_fs, init_male, init_sterile]
time_history = [t]
population_history = [pop]

In [ ]:
while t < T_max:
    # Step 1: Calculate total rate
    T = temperature(t)
    precip = precipitation(t)
    #T = temperature_const(t) # Uncomment this line to use constant temperature
    Hum = 60

    comp = competition1(T, precip)
    #comp = competition2(precip))
    
    prob_M = pop[4]/(pop[4] + pop[5])
    prob_Ms = pop[5]/(pop[4] + pop[5])

    total_rate = (egg_death * pop[0]+
        larva_death * pop[1] +
        pupa_death * pop[2] +
        female_death *(pop[3] + pop[4] + pop[5]) +
        male_death * pop[6] +
        sterile_death * pop[7] +  
        birth * pop[4] +
        transi_el * pop[0] +
        transi_lp * pop[1] +
        transi_pa * pop[2] +
        allee(pop[6], pop[7]) * prob_M * pop[3] +
        allee(pop[6], pop[7]) * prob_Ms * pop[3])
    
    if total_rate == 0:
        print("No life :(")
        break

    # Step 2: Draw time to next event
    time_to_next_event = np.random.exponential(1 / total_rate)
    t += time_to_next_event

    # Step 4: Determine the event
    p = np.random.rand() * total_rate
    s = (egg_death * pop[0]) / total_rate
    if p < (egg_death * pop[0]):
        pop[0] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1]) :
        pop[1] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2]):
        pop[2] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * pop[3]):
        pop[3] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4])):
        pop[4] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5])):
        pop[5] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6]):
        pop[6] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7]):
        pop[7] -= 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4]):
        pop[0] += n_egg
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0]):
        pop[0] -= 1
        pop[1] += 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1]):
        pop[1] -= 1
        pop[2] += 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2]):
        pop[2] -= 1
        pop[6] += 1
    elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2] + allee(pop[6], pop[7]) * prob_M * pop[3]):
        pop[3] -= 1
        pop[4] += 1
    #elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2] + allee(pop[6], pop[7]) * prob_M * pop[3] + allee(pop[6], pop[7]) * prob_Ms * pop[3]):
    #elif p < (egg_death * pop[0] + larva_death * pop[1] + pupa_death * pop[2] + female_death * (pop[3] + pop[4] + pop[5]) + male_death * pop[6] + sterile_death * pop[7] + birth * pop[4] + transi_el * pop[0] + transi_lp * pop[1] + transi_pa * pop[2] + allee(pop[6], pop[7]) * prob_M * pop[3] + allee(pop[6], pop[7]) * prob_Ms * pop[3] + ):
    else :
        pop[3] -= 1
        pop[5] += 1
    
    # Step 3: Sterile insect release
    if int(t) % 7 == 0 and (len(time_history) == 0 or int(time_history[-1]) != int(t))  and pop[3] > 30 and t > 450 and n_rel < 8 and 3 == 4:
        pop[5] += release1(pop[7], 100)
        n_rel += 1
        print("Release no", n_rel, "at time", t, "of", release1(pop[4], 100), "sterile insects")

    # Step 4: Record history
    time_history.append(t)
    population_history.append((pop[0], pop[1], pop[2], pop[3], pop[4], pop[5]))
    if sum(pop) == 0 or pop[4] + pop[5] == 0:
        print("No life :(")
        break
    
    if min(pop) < 0:
        print("Negative population :(")
        break


## Only Larva

Simplified aquatic state with only larva stage with updated rates.

In [ ]:
#modified rates for the model

## Only adults

Simplest model, where only adult mosquitoes are modelled. It depends on the previous rates and functions, with a square root of the female populatoin in birth.

$\beta_A(A) = \displaystyle \frac{-(\delta_L + \tau_L) + \sqrt{(\delta_L + \tau_L)^2 + 4c \beta(A) A}}{2c}$

In [ ]:
# modified rates for the model

In [ ]:
while t < T_max:
    # Step 1: Calculate total rate
    T = temperature(t)
    precip = precipitation(t)
    #T = temperature_const(t) # Uncomment this line to use constant temperature
    Hum = 60

    comp = competition1(T, precip)
    #comp = competition2(precip))
    
    prob_M = pop[4]/(pop[4] + pop[5])
    prob_Ms = pop[5]/(pop[4] + pop[5])





    
    # Step 3: Sterile insect release
    if int(t) % 7 == 0 and (len(time_history) == 0 or int(time_history[-1]) != int(t))  and pop[2] > 30 and t > 450 and n_rel < 8 and 3 == 4:
        pop[5] += release1(pop[4], 100)
        n_rel += 1
        print("Release no", n_rel, "at time", t, "of", release1(pop[4], 100), "sterile insects")

    # Step 5: Record history
    time_history.append(t)
    population_history.append((pop[0], pop[1], pop[2], pop[3], pop[4], pop[5]))
    if sum(pop) == 0 or pop[4] + pop[5] == 0:
        print("No life :(")
        break
    
    if min(pop) < 0:
        print("Negative population :(")
        break
